In [1]:
!pip install datasets transformers librosa torch torchaudio jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.1 MB/s eta 0:00:00


In [2]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.3 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt-get install -y ffmpeg
!pip install soundfile librosa

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,202 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [4]:
# ============================================================================
# 1. DATASET LOADING
# ============================================================================
from datasets import load_dataset, Audio

# Load dataset
ds = load_dataset("hezarai/common-voice-13-fa")

# Cast audio column to Audio feature with target sampling rate
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

# Now access audio
sample = ds['train'][0]
print(f"Audio array shape: {sample['audio']['array'].shape}")
print(f"Transcript: {sample['sentence']}")
print(f"Sample rate: {sample['audio']['sampling_rate']}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/328M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/329M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/303M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/383M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28024 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10440 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10440 [00:00<?, ? examples/s]

Audio array shape: (54144,)
Transcript: تا حرف پول به میان میآید گوشهایش را تیز میکند.
Sample rate: 16000


In [5]:
# ============================================================================
# 2. VOCABULARY AND FEATURE EXTRACTION - FIXED!
# ============================================================================
import torch
import torch.nn as nn
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
import numpy as np
import re

# Parameters
SAMPLE_RATE = 16000
N_MELS = 80
N_FFT = 400
HOP_LENGTH = 160

# 1. Create CLEAN Persian vocabulary
def create_vocab(dataset):
    """Create clean Persian character vocabulary"""
    vocab = set()

    print("Building vocabulary...")
    for item in dataset['train']:
        text = item['sentence']
        # Keep only Persian characters, spaces, and essential punctuation
        text = re.sub(r'[^\u0600-\u06FF\s\.\!\?،]', '', text)
        vocab.update(text)

    # Sort and create mapping
    vocab = sorted(list(vocab))
    char_to_idx = {char: idx + 1 for idx, char in enumerate(vocab)}  # 0 reserved for blank
    char_to_idx['<blank>'] = 0
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}

    print(f"Vocabulary size: {len(char_to_idx)}")
    print(f"Sample characters: {list(char_to_idx.keys())[1:20]}")

    # Verify no English characters
    english_chars = [c for c in char_to_idx.keys() if c.isascii() and c.isalpha()]
    if english_chars:
        print(f"WARNING: Found English characters: {english_chars}")
    else:
        print("✓ Vocabulary is clean (no English characters)")

    return char_to_idx, idx_to_char

char_to_idx, idx_to_char = create_vocab(ds)

# 2. Feature extraction function (unchanged)
def extract_log_mel(audio_array):
    """Extract log mel spectrogram features"""
    # Convert to tensor
    if not isinstance(audio_array, torch.Tensor):
        audio_array = torch.tensor(audio_array, dtype=torch.float32)

    # Extract mel spectrogram
    mel_spec_transform = T.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )

    mel_spec = mel_spec_transform(audio_array)

    # Convert to log scale
    log_mel = torch.log(mel_spec + 1e-9)

    return log_mel

# Test feature extraction
test_audio = ds['train'][0]['audio']['array']
test_features = extract_log_mel(test_audio)
print(f"\nLog mel shape: {test_features.shape}")

Building vocabulary...
Vocabulary size: 58
Sample characters: ['!', '.', '،', '؛', '؟', 'ء', 'آ', 'أ', 'ؤ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د']
✓ Vocabulary is clean (no English characters)

Log mel shape: torch.Size([80, 339])


In [6]:
# ============================================================================
# 3. DATASET AND COLLATE - FIXED TEXT ENCODING
# ============================================================================

def clean_text(text):
    """Clean text to only include characters in vocabulary"""
    # Remove non-Persian characters
    text = re.sub(r'[^\u0600-\u06FF\s\.\!\?،]', '', text)
    # Normalize spaces
    text = ' '.join(text.split())
    return text

class ASRDataset(Dataset):
    def __init__(self, hf_dataset, char_to_idx):
        self.dataset = hf_dataset
        self.char_to_idx = char_to_idx

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # Extract features
        audio = item['audio']['array']
        log_mel = extract_log_mel(audio)

        # Clean and encode text
        text = clean_text(item['sentence'])
        labels = [self.char_to_idx[char] for char in text if char in self.char_to_idx]

        return log_mel.T, torch.tensor(labels, dtype=torch.long)  # Transpose to (time, n_mels)

# Collate function (unchanged)
def collate_fn(batch):
    features, labels = zip(*batch)

    # Pad features
    feature_lengths = torch.tensor([f.size(0) for f in features])
    features_padded = nn.utils.rnn.pad_sequence(features, batch_first=True)

    # Pad labels
    label_lengths = torch.tensor([len(l) for l in labels])
    labels_padded = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)

    return features_padded, labels_padded, feature_lengths, label_lengths

In [7]:
# ============================================================================
# 4. MODEL (unchanged)
# ============================================================================

class CNNCTC(nn.Module):
    def __init__(self, n_mels=80, vocab_size=100):
        super().__init__()

        # CNN layers - Pool only in frequency dimension, not time
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d((1, 2)),  # Only pool frequency: (time, freq) -> (time, freq/2)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d((1, 2)),  # Only pool frequency: (time, freq/2) -> (time, freq/4)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
        )

        # Calculate flattened size (only frequency is reduced by 4)
        self.rnn_input_size = 128 * (n_mels // 4)

        # RNN layers
        self.rnn = nn.LSTM(
            self.rnn_input_size,
            256,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.2
        )

        # Output layer
        self.fc = nn.Linear(512, vocab_size)

    def forward(self, x):
        # x: (batch, time, n_mels)
        x = x.unsqueeze(1)  # (batch, 1, time, n_mels)
        x = self.conv(x)  # (batch, 128, time, n_mels')

        # Reshape for RNN
        batch, channels, time, freq = x.size()
        x = x.permute(0, 2, 1, 3)  # (batch, time, channels, freq)
        x = x.reshape(batch, time, channels * freq)

        # RNN
        x, _ = self.rnn(x)

        # Output
        x = self.fc(x)

        return x.log_softmax(2)

In [8]:
# ============================================================================
# 5. CREATE DATALOADERS
# ============================================================================

print("\nCreating datasets...")
train_dataset = ASRDataset(ds['train'], char_to_idx)
val_dataset = ASRDataset(ds['validation'], char_to_idx)

train_loader = DataLoader(
    train_dataset,
    batch_size=8,  # Start small due to memory
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2
)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")


Creating datasets...
Training batches: 3503
Validation batches: 1305


In [9]:
# ============================================================================
# 6. INITIALIZE MODEL
# ============================================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

model = CNNCTC(n_mels=N_MELS, vocab_size=len(char_to_idx))
model = model.to(device)

criterion = nn.CTCLoss(blank=0, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

print("✓ Model initialized and ready for training!")
print(f"✓ Vocab size: {len(char_to_idx)}")


Using device: cuda
✓ Model initialized and ready for training!
✓ Vocab size: 58


In [10]:
# ============================================================================
# 7. TRAINING FUNCTIONS (unchanged)
# ============================================================================

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0

    for batch_idx, (features, labels, feature_lengths, label_lengths) in enumerate(loader):
        features = features.to(device)
        labels = labels.to(device)
        feature_lengths = feature_lengths.to('cpu')  # CTC needs lengths on CPU
        label_lengths = label_lengths.to('cpu')

        optimizer.zero_grad()

        # Forward pass
        outputs = model(features)  # (batch, time, vocab)
        outputs = outputs.permute(1, 0, 2)  # (time, batch, vocab) for CTC

        # CTC loss
        loss = criterion(outputs, labels, feature_lengths, label_lengths)

        # Check for invalid loss
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Warning: Invalid loss at batch {batch_idx}")
            continue

        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 50 == 0:
            print(f"Batch {batch_idx}/{len(loader)}, Loss: {loss.item():.4f}")

    return total_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for features, labels, feature_lengths, label_lengths in loader:
            features = features.to(device)
            labels = labels.to(device)
            feature_lengths = feature_lengths.to('cpu')
            label_lengths = label_lengths.to('cpu')

            # Forward pass
            outputs = model(features)
            outputs = outputs.permute(1, 0, 2)

            # CTC loss
            loss = criterion(outputs, labels, feature_lengths, label_lengths)

            if not (torch.isnan(loss) or torch.isinf(loss)):
                total_loss += loss.item()

    return total_loss / len(loader)

In [11]:
# ============================================================================
# 8. DECODING - FIXED TO HANDLE CLEAN VOCAB
# ============================================================================

def decode_predictions(outputs, idx_to_char):
    """Greedy CTC decoding with cleaning"""
    # outputs: (batch, time, vocab)
    _, max_indices = torch.max(outputs, dim=2)  # (batch, time)

    decoded_texts = []
    for sequence in max_indices:
        # Remove blanks and repetitions
        prev_idx = None
        decoded = []
        for idx in sequence.cpu().numpy():
            idx = int(idx)  # Convert to Python int
            if idx != 0 and idx != prev_idx:  # 0 is blank
                if idx in idx_to_char:
                    decoded.append(idx_to_char[idx])
            prev_idx = idx
        decoded_texts.append(''.join(decoded))

    return decoded_texts

In [15]:
import re
import unicodedata
from jiwer import wer, cer

def clean_persian_text(text):
    """
    Clean and normalize Persian text for metric calculation
    """
    if not text:
        return ""

    # Normalize Unicode (important for Persian)
    text = unicodedata.normalize('NFKC', text)

    # Replace Arabic/Urdu characters with Persian equivalents
    replacements = {
        'ي': 'ی',  # Arabic yeh to Persian yeh
        'ك': 'ک',  # Arabic kaf to Persian kaf
        'ە': 'ه',  # Arabic heh to Persian heh
        'ے': 'ی',  # Urdu yeh to Persian yeh (this was in your output!)
        '؛': '',   # Arabic semicolon
        '،': '',   # Arabic comma
        '؟': '',   # Arabic question mark
        '.': '',   # Period
        '!': '',   # Exclamation
        '‌': '',   # Zero-width non-joiner
        '‍': '',   # Zero-width joiner
        '\u200c': '',  # ZWNJ
        '\u200d': '',  # ZWJ
    }

    for old, new in replacements.items():
        text = text.replace(old, new)

    # Remove extra whitespace
    text = ' '.join(text.split())

    # Remove any remaining non-Persian characters (keep only Persian letters, digits, spaces)
    # Persian Unicode range: \u0600-\u06FF and some additional characters
    text = re.sub(r'[^\u0600-\u06FF\uFB8A\u067E\u0686\u06AF\u0698\u06A9\u06CC\s\d]+', '', text)

    # Final whitespace cleanup
    text = text.strip()

    return text

def compute_metrics(predictions, references):
    """
    Compute WER and CER metrics with proper Persian text cleaning
    """
    # Clean both predictions and references
    preds_clean = [clean_persian_text(p) for p in predictions]
    refs_clean = [clean_persian_text(r) for r in references]

    # Filter out empty references
    valid_pairs = [(p, r) for p, r in zip(preds_clean, refs_clean) if r.strip() != '']

    if not valid_pairs:
        return {'wer': 1.0, 'cer': 1.0, 'num_samples': 0}

    # IMPORTANT: Convert tuples to lists
    preds, refs = zip(*valid_pairs)
    preds = list(preds)  # Convert from tuple to list
    refs = list(refs)    # Convert from tuple to list

    # Handle empty predictions (replace with single space for metric calculation)
    preds = [p if p.strip() != '' else ' ' for p in preds]

    try:
        wer_score = wer(refs, preds)  # Note: reference first, prediction second
        cer_score = cer(refs, preds)
    except Exception as e:
        print(f"Error computing metrics: {e}")
        print(f"Type of refs: {type(refs)}, Type of preds: {type(preds)}")
        print(f"First ref: {refs[0] if refs else 'empty'}")
        print(f"First pred: {preds[0] if preds else 'empty'}")
        wer_score = 1.0
        cer_score = 1.0

    return {
        'wer': wer_score,
        'cer': cer_score,
        'num_samples': len(valid_pairs)
    }

def evaluate_model(model, loader, device, idx_to_char, num_batches=None):
    """
    Evaluate model on validation set with detailed metrics
    """
    model.eval()
    all_predictions = []
    all_references = []

    with torch.no_grad():
        for batch_idx, (features, labels, _, _) in enumerate(loader):
            if num_batches and batch_idx >= num_batches:
                break

            features = features.to(device)
            outputs = model(features)

            # Decode predictions
            predictions = decode_predictions(outputs, idx_to_char)

            # Decode ground truth
            for i in range(labels.size(0)):
                ref = ''.join([idx_to_char[idx.item()] for idx in labels[i] if idx.item() != 0])
                all_references.append(ref)
                all_predictions.append(predictions[i])

    # Compute metrics
    metrics = compute_metrics(all_predictions, all_references)

    # Add some diagnostic info
    print(f"   Evaluated {metrics['num_samples']} samples")

    # Show cleaned examples
    if len(all_predictions) > 0:
        print(f"\n   Sample after cleaning:")
        pred_clean = clean_persian_text(all_predictions[0])
        ref_clean = clean_persian_text(all_references[0])
        print(f"   Pred: {pred_clean[:80]}")
        print(f"   Ref:  {ref_clean[:80]}")

    return metrics, all_predictions, all_references



In [16]:
# ============================================================================
# 10. TEST INFERENCE BEFORE TRAINING
# ============================================================================

print("\nTesting inference on one batch...")
model.eval()
with torch.no_grad():
    test_features, test_labels, test_feat_len, test_lab_len = next(iter(val_loader))
    test_features = test_features.to(device)
    test_outputs = model(test_features)

    # Decode predictions
    predictions = decode_predictions(test_outputs, idx_to_char)

    # Show first example
    print("\nBefore training - Sample prediction:")
    print(f"Predicted: {predictions[0][:80]}")

    # Ground truth
    ground_truth = ''.join([idx_to_char[idx.item()] for idx in test_labels[0] if idx.item() != 0])
    print(f"Ground truth: {ground_truth[:80]}")

    # Check for any garbage characters
    non_persian = re.findall(r'[A-Za-z]', predictions[0])
    if non_persian:
        print(f"WARNING: Found English characters in prediction: {non_persian}")
    else:
        print("✓ No English characters in prediction")


Testing inference on one batch...

Before training - Sample prediction:
Predicted: بهمیتآن نامرانالیرگرت.ٌے
Ground truth: اهمیت آن نامه را نادیده گرفت.
✓ No English characters in prediction


In [17]:
# ============================================================================
# 11. TRAINING LOOP - WITH BETTER MONITORING
# ============================================================================

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

num_epochs = 13  # Increased to 15 since breakthrough often happens around epoch 8-12
best_val_loss = float('inf')
best_cer = float('inf')

for epoch in range(num_epochs):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{num_epochs}")
    print('='*60)

    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f"\n✓ Train Loss: {train_loss:.4f}")

    # Validate
    val_loss = validate(model, val_loader, criterion, device)
    print(f"✓ Validation Loss: {val_loss:.4f}")

    # Compute WER and CER on validation set (using subset for speed)
    print("\nComputing WER and CER...")
    metrics, preds, refs = evaluate_model(model, val_loader, device, idx_to_char, num_batches=50)

    print(f"✓ WER: {metrics['wer']*100:.2f}%")
    print(f"✓ CER: {metrics['cer']*100:.2f}%")

    # Show sample predictions
    print(f"\nSample predictions (3 examples):")
    for i in range(min(3, len(preds))):
        print(f"\n[{i+1}]")
        print(f"Pred: {preds[i][:100]}")
        print(f"True: {refs[i][:100]}")

        # Check for English characters in predictions
        english_in_pred = re.findall(r'[A-Za-z]', preds[i])
        if english_in_pred:
            print(f"⚠️  WARNING: English chars found: {english_in_pred}")

    # Diagnostic: Check what model is outputting
    model.eval()
    with torch.no_grad():
        test_batch = next(iter(val_loader))
        test_features = test_batch[0][:1].to(device)
        test_output = model(test_features)
        prediction_indices = test_output.argmax(dim=-1).squeeze()

        unique_tokens = torch.unique(prediction_indices).tolist()
        blank_ratio = (prediction_indices == 0).float().mean().item()

        print(f"\n📊 Diagnostics:")
        print(f"   Unique tokens in prediction: {len(unique_tokens)}/{len(char_to_idx)}")
        print(f"   Blank token ratio: {blank_ratio*100:.1f}%")
        if blank_ratio > 0.95:
            print(f"   ⚠️  Model mostly producing blanks - learning in progress")
        elif blank_ratio < 0.50:
            print(f"   ✓ Model producing diverse tokens - breakthrough imminent!")

    # Save best model based on CER
    if metrics['cer'] < best_cer:
        best_cer = metrics['cer']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'wer': metrics['wer'],
            'cer': metrics['cer'],
            'char_to_idx': char_to_idx,
            'idx_to_char': idx_to_char
        }, 'best_model.pt')
        print(f"\n💾 Saved best model! (CER: {metrics['cer']*100:.2f}%)")

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print(f"Best CER: {best_cer*100:.2f}%")
print("="*60)

# Final evaluation on full validation set
print("\nFinal evaluation on full validation set...")
final_metrics, _, _ = evaluate_model(model, val_loader, device, idx_to_char)
print(f"Final WER: {final_metrics['wer']*100:.2f}%")
print(f"Final CER: {final_metrics['cer']*100:.2f}%")


STARTING TRAINING

Epoch 1/13
Batch 0/3503, Loss: 1.0110
Batch 50/3503, Loss: 0.8348
Batch 100/3503, Loss: 1.3331
Batch 150/3503, Loss: 0.8374
Batch 200/3503, Loss: 0.9033
Batch 250/3503, Loss: 0.9584
Batch 300/3503, Loss: 0.9261
Batch 350/3503, Loss: 0.9659
Batch 400/3503, Loss: 1.0686
Batch 450/3503, Loss: 0.9697
Batch 500/3503, Loss: 0.9011
Batch 550/3503, Loss: 0.8020
Batch 600/3503, Loss: 1.2037
Batch 650/3503, Loss: 1.1359
Batch 700/3503, Loss: 1.2275
Batch 750/3503, Loss: 0.8350
Batch 800/3503, Loss: 0.8070
Batch 850/3503, Loss: 0.8213
Batch 900/3503, Loss: 0.8686
Batch 950/3503, Loss: 0.9465
Batch 1000/3503, Loss: 0.9270
Batch 1050/3503, Loss: 1.2729
Batch 1100/3503, Loss: 1.1114
Batch 1150/3503, Loss: 1.3103
Batch 1200/3503, Loss: 0.9529
Batch 1250/3503, Loss: 0.8968
Batch 1300/3503, Loss: 1.2003
Batch 1350/3503, Loss: 0.8671
Batch 1400/3503, Loss: 0.7322
Batch 1450/3503, Loss: 1.1522
Batch 1500/3503, Loss: 0.9929
Batch 1550/3503, Loss: 0.7417
Batch 1600/3503, Loss: 0.8759
Ba

In [18]:
# After training completes, evaluate on test set
print("\n" + "="*60)
print("FINAL EVALUATION ON TEST SET")
print("="*60)

# Create test dataset and loader
print("\nCreating test dataset...")
test_dataset = ASRDataset(ds['test'], char_to_idx)
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2
)

print(f"Test batches: {len(test_loader)}")

# Load best model
print("\nLoading best model...")
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Best model from epoch {checkpoint['epoch']}")
print(f"Validation CER: {checkpoint['cer']*100:.2f}%")

# Evaluate on full test set
print("\nEvaluating on test set...")
test_metrics, test_preds, test_refs = evaluate_model(
    model,
    test_loader,
    device,
    idx_to_char,
    num_batches=None  # Evaluate on full test set
)

print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"✓ WER: {test_metrics['wer']*100:.2f}%")
print(f"✓ CER: {test_metrics['cer']*100:.2f}%")
print(f"✓ Samples evaluated: {test_metrics['num_samples']}")

# Show some example predictions
print("\n" + "="*60)
print("SAMPLE PREDICTIONS FROM TEST SET")
print("="*60)

for i in range(10):
    print(f"\n[{i+1}]")
    print(f"Pred: {test_preds[i]}")
    print(f"True: {test_refs[i]}")
    print(f"Pred (clean): {clean_persian_text(test_preds[i])}")
    print(f"True (clean): {clean_persian_text(test_refs[i])}")

print("\n" + "="*60)
print("EVALUATION COMPLETE!")
print("="*60)


FINAL EVALUATION ON TEST SET

Creating test dataset...
Test batches: 1305

Loading best model...
Best model from epoch 11
Validation CER: 33.61%

Evaluating on test set...
   Evaluated 10440 samples

   Sample after cleaning:
   Pred: جاودتو حس م سافریی
   Ref:  اتوبوس مسافری

TEST SET RESULTS
✓ WER: 90.77%
✓ CER: 38.28%
✓ Samples evaluated: 10440

SAMPLE PREDICTIONS FROM TEST SET

[1]
Pred: جاودتو حس م سافری.؟ے
True: اتوبوس مسافری
Pred (clean): جاودتو حس م سافریی
True (clean): اتوبوس مسافری

[2]
Pred: ام اکاس مسبت دهید و حتبا رراخشید.ے
True: انعکاس مثبت دهید و اعتبار ببخشید
Pred (clean): ام اکاس مسبت دهید و حتبا رراخشیدی
True (clean): انعکاس مثبت دهید و اعتبار ببخشید

[3]
Pred: خراده؟ے
True: جنگ افزارهای ساده
Pred (clean): خرادهی
True (clean): جنگ افزارهای ساده

[4]
Pred: آیا حز رتعالی مرا س دازقید.؟ے
True: آیا حضرت عالی مرا صدا زدید؟
Pred (clean): آیا حز رتعالی مرا س دازقیدی
True (clean): آیا حضرت عالی مرا صدا زدید

[5]
Pred: بی اچحرفت بزنم.ے
True: باید باهاش حرف بزنم
Pred (clean): 